In [1]:
from skimage import data
import napari
from typing_extensions import Annotated
import cv2
%gui qt

ModuleNotFoundError: No module named 'skimage'

In [5]:
def threshold_image(layer: napari.layers.Image, threshold: Annotated[int, {"widget_type" : "IntSlider", "min" : 0, "max" : 255}] = 255) -> napari.types.ImageData:
    if layer:
        _, thresholded = cv2.threshold(layer.data, threshold, 255, cv2.THRESH_TOZERO)
        return thresholded
    
def connected_components(layer: napari.layers.Image, viewer: napari.Viewer):
    if layer:
        _, labels, stats, centroids = cv2.connectedComponentsWithStats(layer.data)
        if 'Coin Labels' in viewer.layers:
            viewer.layers.remove('Coin Labels')
            viewer.layers.remove('Centroids')
        centroids[:, [0, 1]] = centroids[:, [1, 0]]
        centroids = centroids[1:]
        coin_areas = stats[:, cv2.CC_STAT_AREA]
        area = {'area' : coin_areas}
        viewer.add_labels(labels, name='Coin Labels', properties=area)
        viewer.add_points(centroids, ndim=2, name='Centroids', face_color='red', size=2)

def refine(layer: napari.layers.Image, viewer: napari.Viewer, closing_element_size: Annotated[int, {"widget_type" : "SpinBox", "min" : 1, "max" : 10}] = 3, opening_element_size: Annotated[int, {"widget_type" : "SpinBox"}] = 3) -> napari.types.ImageData:
    c = closing_element_size
    o = opening_element_size
    if layer:
        opened = cv2.morphologyEx(layer.data, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (o, o)))
        closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (c, c)))
x data.coins()[50:-50, 50:-50]
viewer = napari.Viewer()
viewer.show()
viewer.theme = 'light'
viewer.add_image(image, rgb=False, name='coins')
viewer.window.add_function_widget(name="Threshold", function=threshold_image, area='bottom', magic_kwargs={'auto_call' : True})
viewer.window.add_function_widget(name="Closing", function=refine, area='bottom', magic_kwargs={'call_button' : 'Refine segmentation'})
viewer.window.add_function_widget(name="Label", function=connected_components, area='bottom', magic_kwargs={'call_button' : 'Label image'})